<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql

In [2]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
%sql sqlite:///my_data1.db



In [4]:
import pandas as pd
import sqlite3

# Download the SpaceX data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
spacex_df = pd.read_csv(url)

# Connect to your database
conn = sqlite3.connect('my_data1.db')

# Import with explicit table name (using all lowercase for consistency)
spacex_df.to_sql('spacex_missions', conn, if_exists='replace', index=False)

# Verify import
print("Tables in database:", pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn))

# Close connection
conn.close()

Tables in database:               name
0  spacex_missions


**Note:This below code is added to remove blank rows from table**


In [5]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [6]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such table: SPACEXTBL
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [7]:
%sql SELECT DISTINCT "Launch_Site" FROM spacex_missions ORDER BY "Launch_Site";




 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [8]:
%%sql
SELECT *
FROM spacex_missions
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [9]:
%%sql
SELECT 
    SUM("PAYLOAD_MASS__KG_") AS total_payload_mass_kg,
    ROUND(SUM("PAYLOAD_MASS__KG_") / 1000, 2) AS total_payload_mass_metric_tons
FROM spacex_missions
WHERE "Customer" LIKE '%NASA%CRS%' 
   OR "Customer" LIKE '%NASA%Commercial Resupply%'
   OR "Customer" = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload_mass_kg,total_payload_mass_metric_tons
48213,48.0


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [10]:
%%sql
SELECT 
    "Booster_Version",
    ROUND(AVG("PAYLOAD_MASS__KG_"), 2) AS avg_payload_mass_kg,
    ROUND(AVG("PAYLOAD_MASS__KG_" / 1000), 2) AS avg_payload_mass_metric_tons
FROM spacex_missions
WHERE "Booster_Version" = 'F9 v1.1'
GROUP BY "Booster_Version";

 * sqlite:///my_data1.db
Done.


Booster_Version,avg_payload_mass_kg,avg_payload_mass_metric_tons
F9 v1.1,2928.4,2.6


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [11]:
%%sql
SELECT 
    "Date",
    "Booster_Version",
    "Landing_Outcome"
FROM spacex_missions
WHERE "Landing_Outcome" LIKE '%Ground Pad%Success%'
   OR "Landing_Outcome" = 'Success (ground pad)'
   OR "Landing_Outcome" = 'Success (Ground Pad)'
ORDER BY "Date" ASC
LIMIT 1;

 * sqlite:///my_data1.db
Done.


Date,Booster_Version,Landing_Outcome
2015-12-22,F9 FT B1019,Success (ground pad)


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [12]:
%%sql
SELECT 
    "Booster_Version",
    COUNT(*) AS successful_landings,
    AVG("PAYLOAD_MASS__KG_") AS avg_payload_mass
FROM spacex_missions
WHERE ("Landing_Outcome" LIKE '%Drone%Success%' OR "Landing_Outcome" LIKE '%ASDS%Success%')
  AND "PAYLOAD_MASS__KG_" BETWEEN 4000 AND 6000
GROUP BY "Booster_Version"
ORDER BY successful_landings DESC;

 * sqlite:///my_data1.db
Done.


Booster_Version,successful_landings,avg_payload_mass


### Task 7




##### List the total number of successful and failure mission outcomes


In [13]:
%%sql
SELECT 
    "Mission_Outcome",
    COUNT(*) AS count
FROM spacex_missions
WHERE "Mission_Outcome" IN ('Success', 'Failure')
GROUP BY "Mission_Outcome"
ORDER BY count DESC;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,count
Success,98


### Task 8



##### List all the booster_versions that have carried the maximum payload mass. Use a subquery.


In [14]:
%%sql
SELECT DISTINCT
    "Booster_Version",
    "PAYLOAD_MASS__KG_" AS max_payload_mass
FROM spacex_missions
WHERE "PAYLOAD_MASS__KG_" = (
    SELECT MAX("PAYLOAD_MASS__KG_") 
    FROM spacex_missions
)
ORDER BY "Booster_Version";

 * sqlite:///my_data1.db
Done.


Booster_Version,max_payload_mass
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [15]:
%%sql
SELECT 
    substr("Date", 6, 2) AS month,
    "Booster_Version",
    "Launch_Site",
    "Landing_Outcome"
FROM spacex_missions
WHERE substr("Date", 0, 5) = '2015'
  AND ("Landing_Outcome" LIKE '%Drone Ship%Failure%' 
       OR "Landing_Outcome" LIKE '%ASDS%Failure%')
ORDER BY month;

 * sqlite:///my_data1.db
Done.


month,Booster_Version,Launch_Site,Landing_Outcome


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [16]:
%%sql
SELECT 
    "Landing_Outcome",
    COUNT(*) AS outcome_count
FROM spacex_missions
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
  AND "Landing_Outcome" IS NOT NULL
GROUP BY "Landing_Outcome"
ORDER BY outcome_count DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,outcome_count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


In [18]:
%%sql
SELECT
ROUND(SUM(CASE
WHEN "Landing_Outcome" IN ('Success (drone ship)', 'Success (ground pad)', 'Success (ocean)') THEN 1
ELSE 0
END) * 100.0 / NULLIF(SUM(CASE
WHEN "Landing_Outcome" IN (
    'Success (drone ship)', 'Success (ground pad)', 'Success (ocean)',
    'Failure (drone ship)', 'Failure (ground pad)', 'Failure (ocean)'
) THEN 1 ELSE 0 END), 0), 2) AS success_rate_percentage,

ROUND(SUM(CASE
WHEN "Landing_Outcome" IN ('Failure (drone ship)', 'Failure (ground pad)', 'Failure (ocean)') THEN 1
ELSE 0
END) * 100.0 / NULLIF(SUM(CASE
WHEN "Landing_Outcome" IN (
    'Success (drone ship)', 'Success (ground pad)', 'Success (ocean)',
    'Failure (drone ship)', 'Failure (ground pad)', 'Failure (ocean)'
) THEN 1 ELSE 0 END), 0), 2) AS failure_rate_percentage

FROM spacex_missions
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
AND "Landing_Outcome" IS NOT NULL;




 * sqlite:///my_data1.db
Done.


success_rate_percentage,failure_rate_percentage
61.54,38.46


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
